# Horarios de defensas de tesis

**Alumno:** VASQUEZ RAMOS, Jose Manuel

**Fecha:** 20/05/2025

___

## Contexto académico & objetivo

Con un DataFrame de 15 tesistas y 6 salas disponibles en diferentes franjas, minimiza solapamientos y huecos sin exceder 4 horas continuas de uso por sala.

## Indicaciones clave

Codifica solución como (tesista → sala,franja). Heurística inicial: asignación secuencial. Vecino: mover 1 tesista a otra franja/sala. Reporta calendario final y métricas de huecos.

## Librerías

In [1]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict

## Control de aleatoriedad

In [2]:
random.seed(42)
np.random.seed(42)

## Dataset

In [3]:
df = pd.read_csv("dataset/tesistas.csv")
tesistas = df['TesistaID'].tolist()
franjas = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6']
salas = [f"Sala{i}" for i in range(1, 7)]

df.head()

,TesistaID,F1,F2,F3,F4,F5,F6
0,T01,1,1,1,0,1,1
1,T02,0,0,0,0,1,1
2,T03,1,1,0,0,0,0
3,T04,1,0,0,0,1,0
4,T05,0,0,0,1,0,0


## Funciones

In [4]:
def disponibilidad_tesista(tesista_row):
    """Devuelve franjas disponibles para un tesista."""
    return [franja for franja in franjas if tesista_row[franja] == 1]

def asignacion_inicial(df):
    """Heurística inicial secuencial: asignar a la primera franja/sala disponible."""
    asignacion = {}
    sala_idx = 0
    franja_idx = 0

    for idx, row in df.iterrows():
        disponibles = disponibilidad_tesista(row)
        if not disponibles:
            asignacion[row['TesistaID']] = None  # Sin opción
            continue
        # Buscar la siguiente sala y franja disponible
        franja = disponibles[franja_idx % len(disponibles)]
        sala = salas[sala_idx % len(salas)]
        asignacion[row['TesistaID']] = (sala, franja)
        sala_idx += 1
    return asignacion

def evaluar(asignacion):
    """Calcula métricas: solapamientos, huecos, franjas continuas por sala."""
    uso_sala_franja = defaultdict(list)  # (sala, franja) → [tesistas]
    franjas_sala = defaultdict(set)      # sala → set de franjas usadas

    for tesista, asignado in asignacion.items():
        if asignado:
            sala, franja = asignado
            uso_sala_franja[(sala, franja)].append(tesista)
            franjas_sala[sala].add(franja)

    solapamientos = sum(len(v) - 1 for v in uso_sala_franja.values() if len(v) > 1)

    huecos = 0
    exceso_horas = 0
    franja_index = {f: i for i, f in enumerate(franjas)}

    for sala in salas:
        franjas_usadas = sorted([franja_index[f] for f in franjas_sala[sala]])
        if franjas_usadas:
            bloques = []
            inicio = franjas_usadas[0]
            for i in range(1, len(franjas_usadas)):
                if franjas_usadas[i] != franjas_usadas[i - 1] + 1:
                    bloques.append((inicio, franjas_usadas[i - 1]))
                    inicio = franjas_usadas[i]
            bloques.append((inicio, franjas_usadas[-1]))
            # calcular huecos internos y duración de bloques
            for start, end in bloques:
                duracion = end - start + 1
                if duracion > 4:
                    exceso_horas += duracion - 4
                huecos += duracion - len([i for i in range(start, end + 1) if i in franjas_usadas])

    return solapamientos, huecos, exceso_horas

def get_neighbors(asignacion, df):
    """Genera vecinos moviendo un tesista a otra sala/franja válida."""
    vecinos = []
    for tesista in asignacion:
        actual = asignacion[tesista]
        row = df[df['TesistaID'] == tesista].iloc[0]
        disponibles = disponibilidad_tesista(row)
        for franja in disponibles:
            for sala in salas:
                nueva = asignacion.copy()
                nueva[tesista] = (sala, franja)
                vecinos.append(nueva)
    return vecinos

def hill_climb(df, max_iters=1000):
    actual = asignacion_inicial(df)
    score_actual = evaluar(actual)

    for _ in range(max_iters):
        vecinos = get_neighbors(actual, df)
        mejor = min(vecinos, key=evaluar)
        score_mejor = evaluar(mejor)

        if score_mejor < score_actual:
            actual, score_actual = mejor, score_mejor
        else:
            break

    return actual, score_actual

## Ejecución

In [5]:
asignacion_final, (solapamientos, huecos, exceso) = hill_climb(df)

print("\nCalendario Final de Defensas:")
for tesista, asignado in asignacion_final.items():
    if asignado:
        sala, franja = asignado
        print(f"{tesista}: {sala}, {franja}")
    else:
        print(f"{tesista}: ❌ No asignado")

print(f"\nMétricas:")
print(f"Solapamientos: {solapamientos}")
print(f"Huecos por sala: {huecos}")
print(f"Exceso de uso continuo (>4h): {exceso} franjas")


Calendario Final de Defensas:
T01: Sala5, F1
T02: Sala2, F5
T03: Sala1, F2
T04: Sala1, F5
T05: Sala5, F4
T06: Sala6, F1
T07: Sala1, F4
T08: Sala2, F2
T09: Sala2, F3
T10: Sala4, F1
T11: Sala5, F2
T12: Sala6, F5
T13: Sala1, F1
T14: Sala2, F1
T15: Sala3, F1

Métricas:
Solapamientos: 0
Huecos por sala: 0
Exceso de uso continuo (>4h): 0 franjas
